In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import random
import mofr
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier, plot_tree
from lightgbm import LGBMClassifier, plot_tree #LGBMRanker
import math
import pickle
import statsmodels.api as sm


import xgboost as xgb

In [3]:
seed=1234

# Loading data

In [4]:
import os
os.getcwd()

'C:\\Users\\AZUBA\\Documents\\Data_science\\Kaggle\\8.March ML Mania 2023'

In [5]:
# MMasseyOrdinals=pd.read_csv(os.getcwd()+r'\relevant_data\MMasseyOrdinals.csv')

MNCAATourneyCompactResults=pd.read_csv(os.getcwd()+r'\relevant_data_updated\WNCAATourneyCompactResults.csv')
MNCAATourneyDetailedResults=pd.read_csv(os.getcwd()+r'\relevant_data_updated\WNCAATourneyDetailedResults.csv')

MNCAATourneySeeds=pd.read_csv(os.getcwd()+r'\relevant_data_updated\WNCAATourneySeeds.csv')

MRegularSeasonCompactResults=pd.read_csv(os.getcwd()+r'\relevant_data_updated\WRegularSeasonCompactResults.csv')
MRegularSeasonDetailedResults=pd.read_csv(os.getcwd()+r'\relevant_data_updated\WRegularSeasonDetailedResults.csv')

MTeams=pd.read_csv(os.getcwd()+r'\relevant_data_updated\WTeams.csv')

# Table by table

## MNCAATourneyCompactResults

In [6]:
MNCAATourneyCompactResults.shape

(1516, 8)

In [7]:
MNCAATourneyCompactResults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1998,137,3104,94,3422,46,H,0
1,1998,137,3112,75,3365,63,H,0
2,1998,137,3163,93,3193,52,H,0
3,1998,137,3198,59,3266,45,H,0
4,1998,137,3203,74,3208,72,A,0


## MNCAATourneyDetailedResults

In [8]:
MNCAATourneyDetailedResults.shape

(760, 34)

In [9]:
MNCAATourneyDetailedResults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,138,3124,69,3201,55,N,0,28,57,...,34,3,5,17,19,12,18,4,1,18
1,2010,138,3173,67,3395,66,N,0,23,59,...,27,14,15,18,26,8,8,8,6,22
2,2010,138,3181,72,3214,37,H,0,26,57,...,15,3,8,10,21,4,16,6,4,20
3,2010,138,3199,75,3256,61,H,0,25,63,...,20,17,22,16,21,13,16,5,4,24
4,2010,138,3207,62,3265,42,N,0,24,68,...,26,11,17,16,22,9,10,3,4,12


## MNCAATourneySeeds

In [10]:
MNCAATourneySeeds.shape

(1608, 3)

In [11]:
MNCAATourneySeeds.head()

,Season,Seed,TeamID
0,1998,W01,3330
1,1998,W02,3163
2,1998,W03,3112
3,1998,W04,3301
4,1998,W05,3272


In [12]:
MNCAATourneySeeds['Seed_number']=MNCAATourneySeeds['Seed'].str.extract('(\d+)')

In [13]:
MNCAATourneySeeds['Seed_number']=MNCAATourneySeeds['Seed_number'].apply(int)

In [14]:
MNCAATourneySeeds['Seed_number'].value_counts()

11    104
16    104
1     100
2     100
3     100
4     100
5     100
6     100
7     100
8     100
9     100
10    100
12    100
13    100
14    100
15    100
Name: Seed_number, dtype: int64

In [15]:
seeds=MNCAATourneySeeds[['Season', 'TeamID', 'Seed_number']]

## MRegularSeasonCompactResults

In [16]:
MRegularSeasonCompactResults.shape

(126173, 8)

In [17]:
MRegularSeasonCompactResults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1998,18,3104,91,3202,41,H,0
1,1998,18,3163,87,3221,76,H,0
2,1998,18,3222,66,3261,59,H,0
3,1998,18,3307,69,3365,62,H,0
4,1998,18,3349,115,3411,35,H,0


## MRegularSeasonDetailedResults

In [18]:
MRegularSeasonDetailedResults.shape

(70783, 34)

In [19]:
MRegularSeasonDetailedResults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,11,3103,63,3237,49,H,0,23,54,...,13,6,10,11,27,11,23,7,6,19
1,2010,11,3104,73,3399,68,N,0,26,62,...,21,14,27,14,26,7,20,4,2,27
2,2010,11,3110,71,3224,59,A,0,29,62,...,14,19,23,17,23,8,15,6,0,15
3,2010,11,3111,63,3267,58,A,0,27,52,...,26,16,25,22,22,15,11,14,5,14
4,2010,11,3119,74,3447,70,H,1,30,74,...,17,11,21,21,32,12,14,4,2,14


## MTeams

In [20]:
MTeams.shape

(375, 2)

In [21]:
MTeams.head()

,TeamID,TeamName
0,3101,Abilene Chr
1,3102,Air Force
2,3103,Akron
3,3104,Alabama
4,3105,Alabama A&M


In [22]:
MTeams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375 entries, 0 to 374
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   TeamID    375 non-null    int64 
 1   TeamName  375 non-null    object
dtypes: int64(1), object(1)
memory usage: 6.0+ KB


# Data preparation

In [23]:
def winning_team(WTeamID, LTeamID, Season, DayNum):
    if WTeamID<LTeamID:
        return (1,WTeamID, LTeamID, Season, DayNum)
    else:
        return (0,LTeamID,WTeamID, Season, DayNum)        

In [24]:
target_df=pd.DataFrame(list(MNCAATourneyCompactResults.apply(lambda x: winning_team(x['WTeamID'], x['LTeamID'], x['Season'], x['DayNum']), axis=1)), columns=['Target','TeamID1', 'TeamID2', 'Season', 'DayNum'])

In [25]:
target_df

,Target,TeamID1,TeamID2,Season,DayNum
0,1,3104,3422,1998,137
1,1,3112,3365,1998,137
2,1,3163,3193,1998,137
3,1,3198,3266,1998,137
4,1,3203,3208,1998,137
...,...,...,...,...,...
1511,1,3163,3301,2022,147
1512,1,3257,3276,2022,147
1513,1,3163,3390,2022,151
1514,0,3257,3376,2022,151


## Same season compact results

### By team

In [26]:
MRegularSeasonCompactResults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1998,18,3104,91,3202,41,H,0
1,1998,18,3163,87,3221,76,H,0
2,1998,18,3222,66,3261,59,H,0
3,1998,18,3307,69,3365,62,H,0
4,1998,18,3349,115,3411,35,H,0


In [27]:
def winning_team2(WTeamID, LTeamID, Season, DayNum, WScore, LScore, WLoc):
    
    def revert_home(x):
        if x=='H':
            return 'A'
        elif x=='A':
            return 'H'
        else:
            return x
        
    if WTeamID<LTeamID:
        return (1,WTeamID, LTeamID, Season, DayNum, WScore, LScore, WLoc)
    else:
        return (0,LTeamID,WTeamID, Season, DayNum, LScore, WScore, revert_home(WLoc))     


In [28]:
reg_season=pd.DataFrame(list(MRegularSeasonCompactResults.apply(lambda x: winning_team2(x['WTeamID'], x['LTeamID'], x['Season'], x['DayNum'], x['WScore'], x['LScore'], x['WLoc']), axis=1)), columns=['Win','TeamID1', 'TeamID2', 'Season', 'DayNum', 'Score1', 'Score2', 'WLoc'])

In [29]:
reg_season

,Win,TeamID1,TeamID2,Season,DayNum,Score1,Score2,WLoc
0,1,3104,3202,1998,18,91,41,H
1,1,3163,3221,1998,18,87,76,H
2,1,3222,3261,1998,18,66,59,H
3,1,3307,3365,1998,18,69,62,H
4,1,3349,3411,1998,18,115,35,H
...,...,...,...,...,...,...,...,...
126168,0,3125,3179,2023,132,71,89,N
126169,0,3131,3221,2023,132,61,66,H
126170,1,3235,3400,2023,132,61,51,N
126171,1,3284,3406,2023,132,80,55,A


In [30]:
reg_season_by_team_win=MRegularSeasonCompactResults.groupby(['Season', 'WTeamID']).agg({'NumOT':'count','WScore': 'sum'}).reset_index().rename(columns=
                                                                                                                  {'WTeamID':'TeamID','NumOT':'Wins', 'WScore':'Points'})

reg_season_by_team_loss=MRegularSeasonCompactResults.groupby(['Season', 'LTeamID']).agg({'NumOT':'count','LScore': 'sum'}).reset_index().rename(columns=
{'LTeamID':'TeamID','NumOT':'Losses', 'LScore':'Points'})


In [31]:
reg_season_by_team=reg_season_by_team_win.merge(reg_season_by_team_loss, how='left', on=['Season','TeamID'])

In [32]:
reg_season_by_team['Matches']=reg_season_by_team['Wins']+reg_season_by_team['Losses']
reg_season_by_team['Points']=reg_season_by_team['Points_x']+reg_season_by_team['Points_y']
reg_season_by_team['Avg_points_per_game']=reg_season_by_team['Points']/reg_season_by_team['Matches']

reg_season_by_team=reg_season_by_team[['Season','TeamID','Wins','Losses','Matches','Points','Avg_points_per_game']]

In [33]:
reg_season_by_team

,Season,TeamID,Wins,Losses,Matches,Points,Avg_points_per_game
0,1998,3102,4,20.0,24.0,1375.0,57.291667
1,1998,3103,11,18.0,29.0,2008.0,69.241379
2,1998,3104,21,9.0,30.0,2297.0,76.566667
3,1998,3106,6,15.0,21.0,1286.0,61.238095
4,1998,3108,12,11.0,23.0,1560.0,67.826087
...,...,...,...,...,...,...,...
8732,2023,3473,1,24.0,25.0,1366.0,54.640000
8733,2023,3474,5,21.0,26.0,1485.0,57.115385
8734,2023,3475,9,17.0,26.0,1622.0,62.384615
8735,2023,3476,8,20.0,28.0,1657.0,59.178571


### By pairs of teams

In [34]:
reg_season.head()

,Win,TeamID1,TeamID2,Season,DayNum,Score1,Score2,WLoc
0,1,3104,3202,1998,18,91,41,H
1,1,3163,3221,1998,18,87,76,H
2,1,3222,3261,1998,18,66,59,H
3,1,3307,3365,1998,18,69,62,H
4,1,3349,3411,1998,18,115,35,H


In [35]:
reg_season=reg_season.groupby(['TeamID1','TeamID2','Season'])['Win'].agg(['count',sum, np.mean]).reset_index().rename(columns={
    'count': 'together_matches_played',
    'sum':'together_number_of_wins',
    'mean': 'together_winning_percentage'
})

## Putting it all together

In [36]:
target_df.head()

,Target,TeamID1,TeamID2,Season,DayNum
0,1,3104,3422,1998,137
1,1,3112,3365,1998,137
2,1,3163,3193,1998,137
3,1,3198,3266,1998,137
4,1,3203,3208,1998,137


In [37]:
reg_season_by_team.head()

,Season,TeamID,Wins,Losses,Matches,Points,Avg_points_per_game
0,1998,3102,4,20.0,24.0,1375.0,57.291667
1,1998,3103,11,18.0,29.0,2008.0,69.241379
2,1998,3104,21,9.0,30.0,2297.0,76.566667
3,1998,3106,6,15.0,21.0,1286.0,61.238095
4,1998,3108,12,11.0,23.0,1560.0,67.826087


In [38]:
reg_season.head()

,TeamID1,TeamID2,Season,together_matches_played,together_number_of_wins,together_winning_percentage
0,3101,3102,2015,1,1,1.0
1,3101,3106,2023,1,1,1.0
2,3101,3114,2022,1,1,1.0
3,3101,3116,2018,1,0,0.0
4,3101,3116,2019,1,0,0.0


In [39]:
df=target_df

In [40]:
df=df.merge(reg_season_by_team, how='left', left_on=['Season', 'TeamID1'], right_on=['Season', 'TeamID']).merge(reg_season_by_team, how='left', left_on=['Season', 'TeamID2'], right_on=['Season', 'TeamID'])

In [41]:
df=df.drop(columns=['TeamID_x', 'TeamID_y'])

In [42]:
df=df.merge(reg_season, how='left', on=['Season', 'TeamID1','TeamID2'])

In [43]:
df=df.merge(seeds, how='left', left_on=['Season', 'TeamID1'], right_on=['Season', 'TeamID']).merge(seeds, how='left', left_on=['Season', 'TeamID2'], right_on=['Season', 'TeamID'])

In [44]:
df=df.drop(columns=['TeamID_x', 'TeamID_y'])

In [45]:
df.columns=[x.replace('_x', '_1').replace('_y','_2') for x in df.columns]

In [46]:
df

,Target,TeamID1,TeamID2,Season,DayNum,Wins_1,Losses_1,Matches_1,Points_1,Avg_points_per_game_1,Wins_2,Losses_2,Matches_2,Points_2,Avg_points_per_game_2,together_matches_played,together_number_of_wins,together_winning_percentage,Seed_number_1,Seed_number_2
0,1,3104,3422,1998,137,21,9.0,30.0,2297.0,76.566667,19,8.0,27.0,2010.0,74.444444,NaN,NaN,NaN,2,15
1,1,3112,3365,1998,137,21,6.0,27.0,2116.0,78.370370,22,6.0,28.0,2144.0,76.571429,NaN,NaN,NaN,3,14
2,1,3163,3193,1998,137,30,2.0,32.0,2696.0,84.250000,20,8.0,28.0,1950.0,69.642857,NaN,NaN,NaN,2,15
3,1,3198,3266,1998,137,28,1.0,29.0,2337.0,80.586207,20,6.0,26.0,1907.0,73.346154,NaN,NaN,NaN,7,10
4,1,3203,3208,1998,137,17,9.0,26.0,1818.0,69.923077,17,9.0,26.0,1996.0,76.769231,NaN,NaN,NaN,10,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1511,1,3163,3301,2022,147,25,5.0,30.0,2243.0,74.766667,29,3.0,32.0,2444.0,76.375000,NaN,NaN,NaN,2,1
1512,1,3257,3276,2022,147,25,4.0,29.0,2093.0,72.172414,22,6.0,28.0,2047.0,73.107143,1.0,1.0,1.0,1,3
1513,1,3163,3390,2022,151,25,5.0,30.0,2243.0,74.766667,28,3.0,31.0,2287.0,73.774194,NaN,NaN,NaN,2,1
1514,0,3257,3376,2022,151,25,4.0,29.0,2093.0,72.172414,29,2.0,31.0,2210.0,71.290323,NaN,NaN,NaN,1,1


In [47]:
df['Avg_points_difference']=df['Avg_points_per_game_1']-df['Avg_points_per_game_2']
df['Avg_points_ratio']=df['Avg_points_per_game_1']/df['Avg_points_per_game_2']

df['Wins_difference']=df['Wins_1']/df['Wins_2']
df['Wins_percentage_ratio']=(df['Wins_1']*df['Matches_2'])/(df['Wins_2']*df['Matches_1'])

df['Seeds_difference']=df['Seed_number_1']-df['Seed_number_2']
df['Seeds_ratio']=df['Seed_number_1']/df['Seed_number_2']

In [48]:
df

,Target,TeamID1,TeamID2,Season,DayNum,Wins_1,Losses_1,Matches_1,Points_1,Avg_points_per_game_1,...,together_number_of_wins,together_winning_percentage,Seed_number_1,Seed_number_2,Avg_points_difference,Avg_points_ratio,Wins_difference,Wins_percentage_ratio,Seeds_difference,Seeds_ratio
0,1,3104,3422,1998,137,21,9.0,30.0,2297.0,76.566667,...,NaN,NaN,2,15,2.122222,1.028507,1.105263,0.994737,-13,0.133333
1,1,3112,3365,1998,137,21,6.0,27.0,2116.0,78.370370,...,NaN,NaN,3,14,1.798942,1.023494,0.954545,0.989899,-11,0.214286
2,1,3163,3193,1998,137,30,2.0,32.0,2696.0,84.250000,...,NaN,NaN,2,15,14.607143,1.209744,1.500000,1.312500,-13,0.133333
3,1,3198,3266,1998,137,28,1.0,29.0,2337.0,80.586207,...,NaN,NaN,7,10,7.240053,1.098711,1.400000,1.255172,-3,0.700000
4,1,3203,3208,1998,137,17,9.0,26.0,1818.0,69.923077,...,NaN,NaN,10,7,-6.846154,0.910822,1.000000,1.000000,3,1.428571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1511,1,3163,3301,2022,147,25,5.0,30.0,2243.0,74.766667,...,NaN,NaN,2,1,-1.608333,0.978942,0.862069,0.919540,1,2.000000
1512,1,3257,3276,2022,147,25,4.0,29.0,2093.0,72.172414,...,1.0,1.0,1,3,-0.934729,0.987214,1.136364,1.097179,-2,0.333333
1513,1,3163,3390,2022,151,25,5.0,30.0,2243.0,74.766667,...,NaN,NaN,2,1,0.992473,1.013453,0.892857,0.922619,1,2.000000
1514,0,3257,3376,2022,151,25,4.0,29.0,2093.0,72.172414,...,NaN,NaN,1,1,0.882091,1.012373,0.862069,0.921522,0,1.000000


In [49]:
df.to_csv('Dataset_for_modelling_female.csv', index=False)

## Dataset for scoring

In [50]:
result=[
]

for x in [2023]:#[2017,2018,2019,2021,2022]
    for y in MTeams['TeamID']:
        for z in MTeams['TeamID']:
            if y<z:
                result.append((x,y,z))

scoring=pd.DataFrame(result, columns=['Season', 'TeamID1', 'TeamID2'])

In [51]:
scoring=scoring.merge(reg_season_by_team, how='left', left_on=['Season', 'TeamID1'], right_on=['Season', 'TeamID']).merge(reg_season_by_team, how='left', left_on=['Season', 'TeamID2'], right_on=['Season', 'TeamID'])

In [52]:
scoring=scoring.drop(columns=['TeamID_x', 'TeamID_y'])

In [53]:
scoring=scoring.merge(reg_season, how='left', on=['Season', 'TeamID1','TeamID2'])

In [54]:
scoring=scoring.merge(seeds, how='left', left_on=['Season', 'TeamID1'], right_on=['Season', 'TeamID']).merge(seeds, how='left', left_on=['Season', 'TeamID2'], right_on=['Season', 'TeamID'])

In [55]:
scoring=scoring.drop(columns=['TeamID_x', 'TeamID_y'])

In [56]:
scoring.columns=[x.replace('_x', '_1').replace('_y','_2') for x in scoring.columns]

In [57]:
scoring['Avg_points_difference']=scoring['Avg_points_per_game_1']-scoring['Avg_points_per_game_2']
scoring['Avg_points_ratio']=scoring['Avg_points_per_game_1']/scoring['Avg_points_per_game_2']

scoring['Wins_difference']=scoring['Wins_1']/scoring['Wins_2']
scoring['Wins_percentage_ratio']=(scoring['Wins_1']*scoring['Matches_2'])/(scoring['Wins_2']*scoring['Matches_1'])

scoring['Seeds_difference']=scoring['Seed_number_1']-scoring['Seed_number_2']
scoring['Seeds_ratio']=scoring['Seed_number_1']/scoring['Seed_number_2']

In [58]:
scoring

,Season,TeamID1,TeamID2,Wins_1,Losses_1,Matches_1,Points_1,Avg_points_per_game_1,Wins_2,Losses_2,...,together_number_of_wins,together_winning_percentage,Seed_number_1,Seed_number_2,Avg_points_difference,Avg_points_ratio,Wins_difference,Wins_percentage_ratio,Seeds_difference,Seeds_ratio
0,2023,3101,3102,12.0,15.0,27.0,1886.0,69.851852,11.0,18.0,...,NaN,NaN,NaN,NaN,6.575990,1.103926,1.090909,1.171717,NaN,NaN
1,2023,3101,3103,12.0,15.0,27.0,1886.0,69.851852,15.0,13.0,...,NaN,NaN,NaN,NaN,2.851852,1.042565,0.800000,0.829630,NaN,NaN
2,2023,3101,3104,12.0,15.0,27.0,1886.0,69.851852,20.0,10.0,...,NaN,NaN,NaN,10.0,-0.014815,0.999788,0.600000,0.666667,NaN,NaN
3,2023,3101,3105,12.0,15.0,27.0,1886.0,69.851852,14.0,15.0,...,NaN,NaN,NaN,NaN,13.058748,1.229935,0.857143,0.920635,NaN,NaN
4,2023,3101,3106,12.0,15.0,27.0,1886.0,69.851852,15.0,15.0,...,1.0,1.0,NaN,NaN,7.151852,1.114065,0.800000,0.888889,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70120,2023,3474,3476,5.0,21.0,26.0,1485.0,57.115385,8.0,20.0,...,NaN,NaN,NaN,NaN,-2.063187,0.965136,0.625000,0.673077,NaN,NaN
70121,2023,3474,3477,5.0,21.0,26.0,1485.0,57.115385,13.0,19.0,...,NaN,NaN,NaN,NaN,-7.509615,0.883797,0.384615,0.473373,NaN,NaN
70122,2023,3475,3476,9.0,17.0,26.0,1622.0,62.384615,8.0,20.0,...,NaN,NaN,NaN,NaN,3.206044,1.054176,1.125000,1.211538,NaN,NaN
70123,2023,3475,3477,9.0,17.0,26.0,1622.0,62.384615,13.0,19.0,...,NaN,NaN,NaN,NaN,-2.240385,0.965333,0.692308,0.852071,NaN,NaN


In [59]:
scoring.to_csv('Dataset_for_scoring_female.csv', index=False)